# Classic Computer Science Problems in Python
## Part 2 Section 1: Search Algorithms

Genes are commonly represented as a sequence of characters A, C, G, T where each letter represents a nucleotide. Three nucleotides together form a codon which represents a specific amino acid, and groups of amino acids group together to form proteins. The goal is to search through a gene sequence to find specific codons in an efficient manner.




In [1]:
# In the example, we use IntEnum because it gives us comparison operators
# such as <, >=, ==, and so on without extra lifting on our part
from enum import IntEnum
from typing import Tuple, List

# Initial setup
Nucleotide : IntEnum = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]
    
gene_str = "ACTGGTCCAGTGGATTACCAGATATTATTATAGGAATACATACTCTACTCGTACGTCGGCTAGACTGATGCCCCGCTCTCATC"
    


In [2]:
# Helper function to convert a gene string into a list of Codons (Nucleotide tuples)
def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    
    # Start at 0, run through the length of s in buckets of 3
    for i in range(0, len(s), 3):
        # stops the algorithm from running over the end of the sequence
        if (i + 2) >= len(s):
            return gene
        
        # Store the three Nucleotides in the current window in a tuple and add to Gene array
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i+1]], Nucleotide[s[i+2]])
        gene.append(codon)
    
    return gene

my_gene : Gene = string_to_gene(gene_str)

### Linear Search Implementation
One of the more simple search methods to find if a codon exists within a Gene or not. Since we will traverse the entire array of tuples, the worst-case time complexity pans out to O(N) time and O(1) space.

In [3]:
def linear_contains(gene: Gene, key_codon: Codon) -> bool:
    # Iterate over all the codon tuples
    for codon in gene:
        # return True if there is a match
        if codon == key_codon:
            return True
    
    # Return false if the key codon hasn't been found in the sequence
    return False

# Test cases
acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
agt: Codon = (Nucleotide.A, Nucleotide.G, Nucleotide.T)
act: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.T)

test_bucket = [acg, gat, agt, act]

# functional linear search
print(linear_contains(my_gene, agt))
print(linear_contains(my_gene, act))

# Since Python built-in sequence types (list, tuple, range) all implement the
# __contains__() method, we can search for a specific item in them by using
# the 'in' operator. The 'in' operator can be used with any type that implements
# a __contains__() method which improves readability and boilerplate
print(acg in my_gene)
print(gat in my_gene)

False
True
False
True


### Binary Search Implementation
Since linear search is bound to having a worst-case runtime at O(N) where N is the number of elements in the array, it is worth exploring other options. One such option is binary search, with the added caveat that it requires the initial array to be sorted ahead of time. Sorting algorithms typically take O(NlogN) time to perform, this may not always be the appropriate search tool.

Binary search works by starting at the middle of the sorted array, comparing it to the target, then reducing the search range by half. Thus, the worst case runtime for binary search sits around O(logN) since the problem space reduces by half for each step in the algorithm.

In situations where you are guaranteed to have a sorted list at all times, this is fantastic. Otherwise, keep in mind that performing a binary search over an unsorted data structure (eg List) will take O(NlogN + logN) => O(NlogN)

In [4]:
def binary_contains(gene: Gene, key_codon: Codon) -> bool:
    # lower bounds on search starts at 0th index
    low: int = 0
    # upper bound on search starts at last index (length - 1)
    high: int = len(gene) - 1
        
    # as long as there is no overlap between low and high
    while low <= high:
        # reduce problem space by half, set as midpoint
        mid: int = (low + high) // 2
        
        # if the codon found at the current middle index is less
        # than the key codon, set the lower bound to the middle + 1
        # We can use these comparison operators because they are 
        # given to us through the Nucleotide IntEnum class
        
        # peek inside by uncommenting the next couple lines
        # print(repr(gene[mid]))
        # print('lower bound: {} mid index: {} upper bound: {} \n'.format(low, mid, high))
        if gene[mid] < key_codon:
            low = mid + 1
        
        # if the codon found at the middle index is more than the target codon
        # set the upper bound to the middle index - 1
        elif gene[mid] > key_codon:
            high = mid - 1
            
        # return True if there is a match
        else:
            return True
    
    # False if no match found 
    return False

sorted_gene = sorted(my_gene)


# binary_contains(sorted_gene, agt)
# binary_contains(sorted_gene, act)
# binary_contains(sorted_gene, acg)
binary_contains(sorted_gene, gat)


True

In [5]:
# Performant Binary Search using the python built-in bisect library
from bisect import bisect_left 

# Returns the index of the codon in the gene sequence if it exists
# otherwise returns a -1 if the codon does not exist in the sequence
def bisect_contains(gene: Gene, key_codon: Codon) -> bool:
    i = bisect_left(gene, key_codon) 
    if i != len(gene) and gene[i] == key_codon: 
        return i 
    else: 
        return -1
    
bisect_contains(sorted_gene, gat)

10

#### Observations
The linear search is outperforming the binary search, and I believe this to be bacause the linear search is able to find the target codon fairly early within the gene sequence and is not approaching the worst-case runtime of O(N). Since there is no way to know beforehand if the target codon is early in the sequence, (without searching), binary search still seems to be the best possible solution for larger gene sequences, especially if we are looking for a rare codon buried deep within the gene sequence.

All said and done, the bisect solution is incredibly fast and consistent. 

In [6]:
# Linear search over unsorted gene array
%timeit for codon in test_bucket: linear_contains(my_gene, codon)
    
# Binary search over sorted gene array
%timeit for codon in test_bucket: binary_contains(sorted_gene, codon)
    
# Binary search with bisect over sorted gene array
%timeit for codon in test_bucket: bisect_contains(sorted_gene, codon)

3.84 µs ± 96.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.98 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.63 µs ± 24.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Generic Search Implementation
The below functions will work on any iterable / key combination


In [7]:
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from typing_extensions import Protocol
from heapq import heappush, heappop

T = TypeVar('T')

# Generic implementation for linear_contians
def gen_linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
        
    return False


C = TypeVar('C', bound='Comparable')

# inherits from the typing_extensions module and implements 
# a range of comparison operators to be used generically
class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool:
        return self == other
    
    def __lt__(self: C, other: C) -> bool:
        return (not self > other) and self != other
        
    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and self != other
    
    def __le__(self: C, other: C) -> bool:
        return self < other or self == other
    
    def __ge__(self: C, other: C) -> bool:
        return not self < other
  

def gen_binary_contains(iterable: Iterable[T], key: T) -> bool:
    low: int = 0
    high: int = len(iterable) - 1
        
    while low <= high:
        mid: int = (low + high) // 2
            
        if iterable[mid] < key:
            low = mid + 1
        elif iterable[mid] > key:
            high = mid - 1
        else:
            return True
    
    return False

def gen_bisect_contains(iterable: Iterable[T], key: T) -> bool:
    i = bisect_left(iterable, key) 
    if i != len(iterable) and iterable[i] == key: 
        return True
    else: 
        return False
    

In [8]:
print(gen_linear_contains([1, 4, 15, 20, 30, 14, 3, 7], 7))
print(gen_binary_contains(['a', 'c', 'd', 'g', 'l', 'p', 's'], 's'))
print(gen_bisect_contains(['a', 'c', 'd', 'g', 'l', 'p', 's'], 'e'))

True
True
False


## Part 2 Section 2: Maze Solving Algorithms


In [9]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional

import random
from math import sqrt

# By passing the str argument to the Cell Enum class, we can
# have string values rather than integer values for the output
class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"
    
class MazeLocation(NamedTuple):
    row: int
    column: int

class Maze:
    def __init__(self, rows:int = 10, columns: int = 10, sparseness: float = 0.2, 
                 start: MazeLocation = MazeLocation(0,0), 
                 goal: MazeLocation = MazeLocation(9,9)) -> None:
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
            
        # Generates the grid, makes everything empty on first pass
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        
        # Uses sparseness parameter to create BLOCKED cells in the grid
        self._randomly_fill(rows, columns, sparseness)
        
        # Add start to grid 
        self._grid[start.row][start.column] = Cell.START
        
        # Add goal to grid
        self._grid[goal.row][goal.column] = Cell.GOAL
            
    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                # generates a random float between 0 and 1.0 at each step
                # if float is lower than the sparseness paramter then BLOCK it
                # in theory, if sparseness is set to 0.2, then 20% of the maze should be blocked
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    
    # returns a boolean test for whether or not the agent has made it to the goal
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal

    # Tests the validity of cells above, below, to the right, and to the left
    # stores all valid moves in the locations bucket as MazeLocations (named tuples)
    # Will be useful for DFS and BFS and A* algorithms
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
            
        return locations
    
    # marks the path of the agent through the maze, 
    # then sets the starting marker and the goal marker 
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH

        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    
    # clears the agent path and resets the starting / ending markers
    # allows us to run multiple iterations of the algorithms to see
    # how they might behave differently, especially with A*
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"

        return output
                    
    
maze: Maze = Maze()
print(maze)

SX       X
X X  X    
          
X    XX X 
X     XX  
  X X  X  
      XX  
X    X    
X   X X X 
X        G



### Depth First Search
This is an algorithm that goes as deep as possibly can before backtracking to the last fork in the decision tree (if it runs into a dead-end). Stacks are well suited to DFS algorithms since they adhere to the LIFO principal (last in, first out like a stack of plates). 


While the DFS algorithm is running, it needs to keep track of two data structures: the stack of potential moves (frontier), and the set of states that have already been visited (explored). As long as there are spaces to explore, DFS will continue to check if the agent has met its goal; if it has not, then it will continue exploring and adding those spaces to the list of 'explored' tiles so that it doesn't get caught in a circle. If the frontier is empty, then we have searched the entire problem space and were unable to reach the goal.

In [10]:
# Stack implementation with empty test
# Stack.empty returns true if there's nothing in the stack
# push adds an item to the list (in our case a Node)
# pop removes the last element in the container and returns it
# __repr__ to pretty print the container elements
class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
            
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.pop()
    
    def __repr__(self) -> str:
        return repr(self._container)
    
# State of the node is a MazeLocation named tuple
class Node(Generic[T]):
    # The Optional type hint implies that the node can be something or None
    def __init__(self, state: T, parent: Optional['Node'], 
                 cost: float = 0.0, heuristic: float = 0.0) -> None:
        self.state: T = state
        self.parent: Optional['Node'] = parent
        self.cost: float = cost
        self.heuristic: float = cost
            
    # Allows you to compare two nodes based off of 'cost' 
    # To be used in A* weighted search algorithm
    def __lt__(self, other: 'Node') -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

In [11]:
# Callable type annotations: 
# Callable[[Arg1Type, Arg2Type], ReturnType].
# initial is the agent's starting point, also a MazeLocation named tuple
def dfs(initial: T, goal_test: Callable[[T], bool], 
        successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    
    # Frontier initialized as an instance of Stack
    frontier: Stack[Node[T]] = Stack()
    
    # initial is the maze's starting point, second argument is the Node's parent
    # since the initial Node does not have a parent, we can set this to None
    frontier.push(Node(initial, None))
    
    # Explored: set of named tuples initialized with maze's starting point
    explored: Set[T] = {initial}
        
    
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        # break if we've found the goal
        if goal_test(current_state):
            return current_node
        
        # check for where we can go next
        for child in successors(current_state):
            # skip explored tiles
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    # Returns None if there isn't a path to the goal
    return None
    
def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
        
    # reverse the path list because we are appending from the end to the beginning
    path.reverse()
    return path



In [13]:
dfs_maze: Maze = Maze()

dfs_solution: Optional[Node[MazeLocation]] = dfs(dfs_maze.start, dfs_maze.goal_test, dfs_maze.successors)
if dfs_solution is None:
    print('no solution found')
else:
    path: List[MazeLocation] = node_to_path(dfs_solution)
    dfs_maze.mark(path)
    print(dfs_maze)
    

S*X  X   X
 *X  X  X 
 ***XXX***
 XX*****X*
X   XX  **
 X XX**** 
X    *    
  X  ****X
    X   **
  XXX X XG



### Breadth First Search
Since DFS doesn't usually find the shortest path, BFS tries to solve for the shortest path by systematically going through one layer of nodes for each iteration of the algorithm. The tradeoff here is that DFS will typically be faster to find a solution, but BFS will find a more 'efficient' solution. 


Unlike DFS, Breadth First Search uses a data structure known as Queue which operates on the FIFO (first in, first out) principal. This is more akin to a bathroom line or a queue to buy concert tickets in that the queue elements will be popped sequentially from the first element through the last element in line.


To build the Queue data structure, we start by using an instance of Deque which has access to a popleft method. This allows us to grab elements from the left side of the container in constant O(1) time. Popping from the left on a traditional array has O(n) time complexity where n is the number of array elements that need to be reindexed after removing the 0th element.

In [14]:
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.popleft()
    
    def __repr__(self) -> str:
        return repr(self._container)

In [15]:
def bfs(initial: T, goal_test: Callable[[T], bool], 
        successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    
    # Frontier initialized as an instance of Stack
    frontier: Queue[Node[T]] = Queue()
    
    # initial is the maze's starting point, second argument is the Node's parent
    # since the initial Node does not have a parent, we can set this to None
    frontier.push(Node(initial, None))
    
    # Explored: set of named tuples initialized with maze's starting point
    explored: Set[T] = {initial}
        
    
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        # break if we've found the goal
        if goal_test(current_state):
            return current_node
        
        # check for where we can go next
        for child in successors(current_state):
            # skip explored tiles
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    # Returns None if there isn't a path to the goal
    return None
    



In [16]:
bfs_maze: Maze = Maze()

bfs_solution: Optional[Node[MazeLocation]] = bfs(bfs_maze.start, bfs_maze.goal_test, bfs_maze.successors)
if bfs_solution is None:
    print('no solution found')
else:
    path: List[MazeLocation] = node_to_path(bfs_solution)
    bfs_maze.mark(path)
    print(bfs_maze)

S         
****     X
X X* X X  
XX *      
   * XX   
   *X X X 
 XX*      
   *X  X X
   *******
XX  X  XXG



### A* Search Algorithm
A midpoit between Depth First Search and Breadth First Search is the A* search algorithm. This aims to find the shortest path between the start and the goal by using a combination of a cost function and a heuristic function to focus on pathways most likely to reach the goal quickly.


The cost function g(n) evaluates the cost to get to a particular state. For this particular example, that would be the number of steps that the agent has moved through in order to reach the current state. The Heuristic function h(n) gives an estimate of the cost to reach the goal from the current state. If h(n) is an admissable heuristic, which means that it never overestimates the cost to reach the goal, then the final path will be optimal. On a two dimensional plane, (this maze), a straight-line heuristic can be considered an admissable heuristic since a straight line is the shortest distance between two points.


The total cost for state at any given point in time can be given through f(n) where f(n) = g(n) + h(n). To decide where the agent should move in the frontier, the A* algorithm chooses the option with the lowest f(n) (cost).


A* uses something called a 'priority queue' as its foundational data structure. A priority queue maintains its elements in an internal order such that popping returns the highest priority element (lowest f(n)) at all times. To achieve this, the priority queue uses a binary heap which results in O(lg n) push time and O(lg n) pop time. Python's standard library includes heappush and heappop functions, which we will use to create a PriorityQueue data structure.

In [17]:
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
            
    @property
    def empty(self) -> bool:
        return not self._container
    
    # to determine the priority of an element, heappush and heappop run comparisons using
    # the < operator, which is why we implemented __lt__ on the Node class earlier
    # One Node is compared to another by looking at its cost function g(n) + h(n)
    def push(self, item: T) -> None:
        heappush(self._container, item)
        
    def pop(self) -> T:
        return heappop(self._container)
    
    def __repr__(self) -> str:
        return repr(self._container)

### Heuristics 
#### Euclidean Distance
A heuristic is an intuition around how to solve a particular problem, or better: an educated guess. For example, the Euclidean distance derives the shortest distance between two points by using the pythagorean theorem. To put it in better perspective, the Euclidean distance between two points could be considered 'how a bird flies': it sails above obstacles for the most direct path between to points in space.

#### Manhattan Distance
Since there are obstacles in this maze, a better heuristic for this problem would be the Manhattan Distance. To get anywhere in Manhattan, you must traverse a certain number of horizontal blocks and a certain number of vertical blocks in order to reach your destination - much like in this maze example. The Manhattan distance may be derived by finding the difference in rows between two maze locations and summing it with the distance in columns.

In [18]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance

def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

In [19]:
def astar(initial: T, goal_test: Callable[[T], bool], 
        successors: Callable[[T], List[T]], heuristic: Callable[[T], bool]) -> Optional[Node[T]]:
    
    # Frontier initialized as an instance of Stack
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    
    # initial is the maze's starting point, second argument is the Node's parent
    # since the initial Node does not have a parent, we can set this to None
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    
    explored: Dict[T, float] = {initial: 0.0}

    
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        # break if we've found the goal
        if goal_test(current_state):
            return current_node
        
        # check for where we can go next
        for child in successors(current_state):
            # Adding 1 assumes a grid, for more sophisticated applications
            # you can find a way to derive more sophisticated weighting
            new_cost: float = current_node.cost + 1
            
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    
    # Returns None if there isn't a path to the goal
    return None
    



In [21]:
astar_maze: Maze = Maze()

distance: Callable[[MazeLocation], float] = manhattan_distance(astar_maze.goal)   
    
astar_solution: Optional[Node[MazeLocation]] = astar(astar_maze.start, astar_maze.goal_test, 
                                                     astar_maze.successors, distance)
if astar_solution is None:
    print('no solution found')
else:
    path: List[MazeLocation] = node_to_path(astar_solution)
    astar_maze.mark(path)
    print(astar_maze)

S     X   
* X   X   
**        
X****  XX 
 XXX*    X
    *     
    * X   
    ** XX 
   X ****X
   X  X *G

